In [4]:
import pandas as pd
import requests
import numpy as np
import json
import yaml

In [17]:
with open("/home/srg-dell/ibeems_data/credential/credential_iparkmall.yaml", "r") as f:
    credential = yaml.safe_load(f)

In [18]:
credential

{'serviceToken': 'JJVmAvpUEo',
 'endpoint': 'http://59.6.143.42:28082/sensor/data?fromDateTime={fromDateTime}&toDateTime={toDateTime}&path={path}',
 'metadata_rawdata_path': 'https://docs.google.com/spreadsheets/d/e/2PACX-1vRb4pF3aQv-gHZfuarj4rwJgCGbamU0GLVWCfO7D8zCx-XiwhOVBSZqW-6KmtfUDw/pub?output=xlsx'}

In [19]:

start_time_utc="2025-11-08T00:00:00+00:00"
end_time_utc="2025-11-08T00:01:00+00:00"
start_time=pd.Timestamp(start_time_utc)
end_time=pd.Timestamp(end_time_utc)

tz="Asia/Seoul"
if start_time.tz is None:
    start_time=start_time.tz_localize("UTC").tz_convert(tz).strftime("%Y-%m-%dT%H:%M:%S")
else:
    start_time=start_time.tz_convert(tz).strftime("%Y-%m-%dT%H:%M:%S")

if end_time.tz is None:
    end_time=end_time.tz_localize("UTC").tz_convert(tz).strftime("%Y-%m-%dT%H:%M:%S")
else:
    end_time=end_time.tz_convert(tz).strftime("%Y-%m-%dT%H:%M:%S")


query_start_time=(pd.Timestamp(start_time)-pd.Timedelta("1 h")).strftime("%Y-%m-%dT%H:%M:%S")
query_end_time=(pd.Timestamp(end_time)+pd.Timedelta("1 h")).strftime("%Y-%m-%dT%H:%M:%S")


In [20]:
credential

{'serviceToken': 'JJVmAvpUEo',
 'endpoint': 'http://59.6.143.42:28082/sensor/data?fromDateTime={fromDateTime}&toDateTime={toDateTime}&path={path}',
 'metadata_rawdata_path': 'https://docs.google.com/spreadsheets/d/e/2PACX-1vRb4pF3aQv-gHZfuarj4rwJgCGbamU0GLVWCfO7D8zCx-XiwhOVBSZqW-6KmtfUDw/pub?output=xlsx'}

In [22]:
credential

{'serviceToken': 'JJVmAvpUEo',
 'endpoint': 'http://59.6.143.42:28082/sensor/data?fromDateTime={fromDateTime}&toDateTime={toDateTime}&path={path}',
 'metadata_rawdata_path': 'https://docs.google.com/spreadsheets/d/e/2PACX-1vRb4pF3aQv-gHZfuarj4rwJgCGbamU0GLVWCfO7D8zCx-XiwhOVBSZqW-6KmtfUDw/pub?output=xlsx'}

In [29]:

path='/ys_ipark/1/DE-181/AI-1'

url=credential['endpoint'].format(path=path,fromDateTime=query_start_time,toDateTime=query_end_time)


In [30]:
# try query
payload = {}
headers = {
    'serviceToken': credential['serviceToken']#'XX514lgodZ'
}
print(f'start {url}')
response = requests.get(url, headers=headers, data=payload, timeout=10)

start http://59.6.143.42:28082/sensor/data?fromDateTime=2025-11-08T08:00:00&toDateTime=2025-11-08T10:01:00&path=/ys_ipark/1/DE-181/AI-1


In [33]:
url

'http://59.6.143.42:28082/sensor/data?fromDateTime=2025-11-08T08:00:00&toDateTime=2025-11-08T10:01:00&path=/ys_ipark/1/DE-181/AI-1'

In [31]:
response

<Response [200]>

In [32]:
response.text

''

In [14]:
textData=response.text
if textData is None or textData=="":
    #df_data=None
    raise ValueError(f"No data received for {url}. data is empty {textData}")
    
else:
    
    data_list = json.loads(textData)
    if not data_list:
        raise ValueError(f"No data received for {url}")
    else:
        try_error=False #success

ValueError: No data received for http://lottemart.iptime.org:28082/sensor/data?fromDateTime=2025-11-08T08:00:00&toDateTime=2025-11-08T10:01:00&path=/ys_ipark/1/DE-184/AI-9. data is empty 

In [ ]:


def query_endpoint(url,credential,n_try=2,sleep_time=0.2):
    try_error=True
    n_try_=1
    
    while try_error:
        try:
            # try query
            payload = {}
            headers = {
                'serviceToken': credential['serviceToken']#'XX514lgodZ'
            }
            print(f'start {url}')
            response = requests.get(url, headers=headers, data=payload, timeout=10)
            if response.status_code == 200:
                textData=response.text
                if textData is None or textData=="":
                    #df_data=None
                    raise ValueError(f"No data received for {url}. data is empty {textData}")
                    
                else:
                    
                    data_list = json.loads(textData)
                    if not data_list:
                        raise ValueError(f"No data received for {url}")
                    else:
                        try_error=False #success
                        
            else:
                raise ValueError(f"{url}: Request failed with status code {response.status_code}")
        
        except Exception as e:
            time.sleep(sleep_time+n_try_/10)
            n_try_=n_try_+1
            try_error=True
            print(e)
            if n_try_>=n_try:
                print(f"after {n_try_}, it still failed. the error message is {e}. but, function continues..")
                try_error=False
                data_list=None
    return data_list